In [12]:
import random
import math
def trailing_zeroes(num):
    """Counts the number of trailing 0 bits in num."""
    if num == 0:
        return 2**5#L # Assumes 32 bit integer inputs!
    p = 0
    while (num >> p) & 1 == 0:
        p += 1
    return p

def estimate_cardinality_LL(stream, k):
    num_buckets = 2 ** k
    max_zeroes = [0] * num_buckets
    for h in stream:
        bucket = h & (num_buckets - 1) # Mask out the k least significant bits as bucket ID
        bucket_hash = h >> k
        max_zeroes[bucket] = max(max_zeroes[bucket], trailing_zeroes(bucket_hash))
    
    return 2 ** (float(sum(max_zeroes)) / num_buckets) * num_buckets * 0.79402

def estimate_cardinality_HLL(stream, k):
    num_buckets = 2 ** k
    max_zeroes = [0] * num_buckets
    for h in stream:
        bucket = h & (num_buckets - 1) # Mask out the k least significant bits as bucket ID
        bucket_hash = h >> k
        max_zeroes[bucket] = max(max_zeroes[bucket], trailing_zeroes(bucket_hash)+1)
    
    m = num_buckets
    if m == 16:
        alpha = 0.673
    elif m == 32:
        alpha = 0.697
    elif m == 64:
        alpha = 0.709
    else:
        alpha = 0.7213/(1 + 1.079/m)
            
    #Cardinality
    DV_est = alpha * m**2 * 1./sum([2**-i for i in max_zeroes])
        
     #Corrections:
    if DV_est < 5/2 * m: # small range correction
     
        V = sum([x == 0 for x in max_zeroes])#count_of_zero_registers( registers ) # the number of registers equal to zero
        if V == 0:  # if none of the registers are empty, use the HLL estimate
             DV = DV_est
        else:
             DV = m * math.log10(m/V)  # i.e. balls and bins correction
 
    if DV_est <= ( 1/30 * 2**32 ):  # intermediate range, no correction
          DV = DV_est
    if DV_est > ( 1/30 * 2**32 ):  # large range correction
         DV = -2**32 * math.log10( 1 - DV_est/2**32)
    
    return DV

In [13]:
import numpy as np
def estimate_cardinality_FM(stream, k):
    num_bits = 32 #L
    m = 2**k
    bitmap = np.zeros([m, (num_bits-k)])
    for s in range(len(stream)):
       # s: individual hash value
        bin_bit = "{0:b}".format(stream[s])
      # determine the row index
        rev = bin_bit[::-1]
        idx = rev[0:k][::-1]
        idxi = int(idx, 2)

      # count trailing zeros of the remaining binary's
      # this represents the column index.
        end = k
        try:
            hashv = int(bin_bit[0:-end])
        except ValueError:
            hasv = 0
        idxj = trailing_zeroes(hashv)#, num_bits)

      # assign the index of the bitmap to a 1
        bitmap[idxi, idxj] = 1

    # calculate R as follows:
    #   for every row in the bitmap:
    #   - if it only consists of 1's:
    #       row i = 0
    #   - else:
    #       give the index of where the
    #       first zero

    R = []
    for i in range(m):
      # check which the indexes of each individual row 
      # that is equal to zero.
        r = np.where(bitmap[i, ] == 0)[0]

        # take the smallest index
        r_min = np.amin(r)
        R.append(r_min)   

    phi = 0.77351
    est_count = m/phi * 2**(np.mean(R)) # Probabilistic_Counting error
    return est_count


In [13]:

random.seed(1943)

In [14]:
ks = [4,5,6,7,8,9,10,11,12,13,14,15,16]
ms = [2**k for k in ks]
Ns = [10**3,10**4,10**5,10**6,10**7]
streams = [[random.randint(0,2**32) for i in range (N)] for N in Ns]

In [ ]:
random.seed(1943)
LL = []
HLL = []
FM = []
for loops in range(10): #10
    print("\nLoop:",loops,"\nN:")
    ks = [4,5,6,7,8,9,10,11,12,13,14,15,16]
    ms = [2**k for k in ks]
    Ns = [10**3,10**4,10**5,10**6,5*10**6,10**7,2*10**7]
    #streams = [[random.randint(0,2**32) for i in range (N)] for N in Ns]


    #card = [len(list(set(stream))) for stream in streams]
    errors_LL = []
    errors_HLL = []
    errors_FM = []
    for j in range(len(Ns)):
        print(j, " ")
        stream = [random.randint(0,2**32) for i in range (Ns[j])]
        card = len(list(set(stream)))
        err_ks_LL = [card] + [1] * len(ks)
        err_ks_HLL = [card] + [1] * len(ks)
        err_ks_FM = [card] + [1] * len(ks)

        for i in range(len(ks)):
            card_est = estimate_cardinality_LL(stream,ks[i])   #steams[j]
            err_ks_LL[i+1] = abs(card_est - card) / card #card[j]
        
            card_est = estimate_cardinality_HLL(stream,ks[i]) #steams[j]
            err_ks_HLL[i+1] = abs(card_est - card) / card #card[j]
            
            card_est = estimate_cardinality_FM(stream,ks[i]) #steams[j]
            err_ks_FM[i+1] = abs(card_est - card) / card #card[j]
    
        
        errors_LL.append(err_ks_LL)
        errors_HLL.append(err_ks_HLL)
        errors_FM.append(err_ks_FM)
    LL = LL + errors_LL
    HLL = HLL + errors_HLL
    FM = FM + errors_FM

import csv
with open('HLL_data3.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(HLL)
with open('LL_data3.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(LL)
with open('FM_data3.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(FM)



Loop: 0 
N:
0  
1  
2  
3  
4  


In [36]:
j

6